### 示例选择器 Example selectors

**由于对话上下文的限制，你可能需要在大量的提示词模版中检索适合你的提示词，这时候可以用到示例选择器Example selectors**

**设置环境变量key**

In [12]:
import os
os.environ["OPENAI_API_KEY"] = "sk-xxx"
os.environ["OPENAI_API_BASE"] = "https://api.chatanywhere.tech/v1"

示例选择器分类
- 按长度选择

- 最大边际相关(MMR)选择

-

#### 按长度选择

此示例选择器根据长度选择要使用的示例。当您担心构造一个将遍历上下文窗口长度的提示符时，这非常有用。对于较长的输入，它将选择较少的示例来包含，而对于较短的输入，它将选择更多的示例。

In [6]:
from langchain_core.example_selectors import LengthBasedExampleSelector
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

# 示例列表
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="输入: {input}\n输出: {output}",
)
example_selector = LengthBasedExampleSelector(
    # T可供选择的例子
    examples=examples,
    # 用于格式化示例的 PromptTemplate。
    example_prompt=example_prompt,
    # 格式化示例的最大长度。长度由下面的 get _ text _ Length 函数度量。
    max_length=25,
    # get_text_length: Callable[[str], int] = lambda x: len(re.split("\n| ", x))
)
dynamic_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="给出每个词语的反义词",
    suffix="输入: {adjective}\n输出:",
    input_variables=["adjective"],
)

一个小输入的例子，它选择所有的例子。

In [7]:
print(dynamic_prompt.format(adjective="big"))

给出每个词语的反义词

输入: happy
输出: sad

输入: tall
输出: short

输入: energetic
输出: lethargic

输入: sunny
输出: gloomy

输入: windy
输出: calm

输入: big
输出:


测试长文本的输入时选择的例子，会只选择少量的例子

In [8]:
long_string = "big and huge and massive and large and gigantic and tall and much much much much much bigger than everything else"
print(dynamic_prompt.format(adjective=long_string))

给出每个词语的反义词

输入: happy
输出: sad

输入: big and huge and massive and large and gigantic and tall and much much much much much bigger than everything else
输出:


#### 最大边际相关(MMR)选择

**MaxMarginalRequanceExampleSelector 根据哪些示例与输入最相似的组合来选择示例，同时还对多样性进行优化。它通过找到嵌入的例子，这些例子与输入有最大的余弦距离，然后反复添加它们，同时惩罚它们与已经选择的例子的接近程度。**


安装相应依赖

In [14]:
! pip install langchain_community
! pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 3.6 MB/s eta 0:00:00a 0:00:010m


In [22]:
from langchain_community.vectorstores import FAISS
from langchain_core.example_selectors import (
    MaxMarginalRelevanceExampleSelector,
    SemanticSimilarityExampleSelector,
)
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_openai import OpenAIEmbeddings

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

examples: input_variables=['input', 'output'] template='Input: {input}\nOutput: {output}'


In [17]:
example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    examples,
    # 使用openai的Embedding度量语义相似度的嵌入。
    OpenAIEmbeddings(
      openai_api_base=os.getenv("OPENAI_API_BASE"),
      openai_api_key=os.getenv("OPENAI_API_KEY"),
    ),
    # 用于存储嵌入并执行最近邻搜索的向量数据库
    FAISS,
    # 要生成的示例的数量。
    k=2,
)
mmr_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)

In [19]:
# Input is a feeling, so should select the happy/sad example as the first one
print(mmr_prompt.format(adjective="worried"))

Give the antonym of every input

Input: happy
Output: sad

Input: windy
Output: calm

Input: worried
Output:


In [20]:
# 让我们来比较一下，如果我们只考虑相似性,
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    FAISS,
    k=2,
)
similar_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)
print(similar_prompt.format(adjective="worried"))

Give the antonym of every input

Input: happy
Output: sad

Input: sunny
Output: gloomy

Input: worried
Output:
